In [ ]:
!git clone https://github.com/haotian-liu/LLaVA.git
%cd LLaVA
!pip install -e .

加载模型

In [ ]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

model_path = "liuhaotian/llava-v1.5-7b"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path),
    load_in_4bit=True
)

Github demo

In [ ]:
model_path = "liuhaotian/llava-v1.5-7b"
prompt = "Please determine if this is a photo about computer? Answer yes or no"
image_file = "/content/drive/MyDrive/target_data/computer/电脑_1.png"

args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 512
})()

eval_model(args)


You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


True


初始化参数

In [2]:
import io
import os
from contextlib import redirect_stdout

def get_eval_model_output(args):
    # 创建一个StringIO对象来捕获输出
    buffer = io.StringIO()
    # 重定向标准输出到buffer
    with redirect_stdout(buffer):
        # *****************注意*****************
        # 修改llava/eval/run_llava.py，删除每次的函数加载，节省时间
        eval_model(args, tokenizer, model, image_processor, context_len)
    # 获取buffer中的内容，即eval_model的输出
    output = buffer.getvalue()
    return output

model_path = "liuhaotian/llava-v1.5-7b"
prompt = ""
image_file = ""

args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 512
})()

下面三段代码相同，数据不同，原因是中英文CLIP合并后得到的结果不同


*   target_data：中文CLIP没有卡阈值得到最高的精度
*   target_data2：英文CLIP阈值应为0.24，错选为0.23
*   target_data3：最终结果

In [ ]:
categories = {
    "pet cat": "宠物猫",
    "tomato": "番茄",
    "paper-cut": "剪纸",
    "computer": "电脑",
    "dumpling": "饺子"
}

def calculate_metrics(TP, FP, FN):
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1


def verify_and_calculate_metrics(target_root_folder, categories):
    total_precision, total_recall, total_f1 = 0, 0, 0

    for category, keyword in categories.items():
        TP, FP, FN = 0, 0, 0

        category_folder = os.path.join(target_root_folder, category)
        prompt = f"Please determine if this is a photo about {category}? Answer yes or no"
        args.query = prompt

        for image_name in os.listdir(category_folder):
            image_path = os.path.join(category_folder, image_name)
            args.image_file = image_path

            output = get_eval_model_output(args)
            is_correct = "yes" in output.lower()

            # 根据图片名称判断真实类别
            actual_category = keyword if keyword in image_name else "其他"

            # 更新统计数据
            if is_correct and actual_category == keyword:
                TP += 1
            elif is_correct and actual_category != keyword:
                FP += 1

        # 计算并打印当前类别的Precision、Recall和F1 score
        FN = 200 - TP
        precision, recall, f1 = calculate_metrics(TP, FP, FN)
        total_precision += precision
        total_recall += recall
        total_f1 += f1
        print(f"Category: {category}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    # 计算并打印总体平均Precision、Recall和F1 score
    average_precision = total_precision / len(categories)
    average_recall = total_recall / len(categories)
    average_f1 = total_f1 / len(categories)
    print(f"Average Precision: {average_precision:.4f}, Average Recall: {average_recall:.4f}, Average F1: {average_f1:.4f}")


# 示例调用函数
target_root_folder = "/content/drive/MyDrive/target_data"
verify_and_calculate_metrics(target_root_folder, categories)

Category: pet cat, Precision: 0.9132, Recall: 1.0000, F1 Score: 0.9547
Category: tomato, Precision: 0.9949, Recall: 0.9750, F1 Score: 0.9848
Category: paper-cut, Precision: 0.8609, Recall: 0.9900, F1 Score: 0.9209
Category: computer, Precision: 0.9469, Recall: 0.9800, F1 Score: 0.9631
Category: dumpling, Precision: 0.7976, Recall: 0.9850, F1 Score: 0.8814
Average Precision: 0.9027, Average Recall: 0.9860, Average F1: 0.9410


In [ ]:
categories = {
    "pet cat": "宠物猫",
    "tomato": "番茄",
    "paper-cut": "剪纸",
    "computer": "电脑",
    "dumpling": "饺子"
}

def calculate_metrics(TP, FP, FN):
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1


def verify_and_calculate_metrics(target_root_folder, categories):
    total_precision, total_recall, total_f1 = 0, 0, 0

    for category, keyword in categories.items():
        TP, FP, FN = 0, 0, 0

        category_folder = os.path.join(target_root_folder, category)
        prompt = f"Please determine if this is a photo about {category}? Answer yes or no"
        args.query = prompt

        for image_name in os.listdir(category_folder):
            image_path = os.path.join(category_folder, image_name)
            args.image_file = image_path

            output = get_eval_model_output(args)
            is_correct = "yes" in output.lower()

            # 根据图片名称判断真实类别
            actual_category = keyword if keyword in image_name else "其他"

            # 更新统计数据
            if is_correct and actual_category == keyword:
                TP += 1
            elif is_correct and actual_category != keyword:
                FP += 1

        # 计算并打印当前类别的Precision、Recall和F1 score
        FN = 200 - TP
        precision, recall, f1 = calculate_metrics(TP, FP, FN)
        total_precision += precision
        total_recall += recall
        total_f1 += f1
        print(f"Category: {category}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    # 计算并打印总体平均Precision、Recall和F1 score
    average_precision = total_precision / len(categories)
    average_recall = total_recall / len(categories)
    average_f1 = total_f1 / len(categories)
    print(f"Average Precision: {average_precision:.4f}, Average Recall: {average_recall:.4f}, Average F1: {average_f1:.4f}")


# 示例调用函数
target_root_folder = "/content/drive/MyDrive/target_data2"
verify_and_calculate_metrics(target_root_folder, categories)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.w

Category: pet cat, Precision: 0.9615, Recall: 1.0000, F1 Score: 0.9804
Category: tomato, Precision: 0.9949, Recall: 0.9750, F1 Score: 0.9848
Category: paper-cut, Precision: 0.8603, Recall: 0.9850, F1 Score: 0.9184


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Category: computer, Precision: 0.9897, Recall: 0.9600, F1 Score: 0.9746
Category: dumpling, Precision: 0.8140, Recall: 0.9850, F1 Score: 0.8914
Average Precision: 0.9241, Average Recall: 0.9810, Average F1: 0.9499


In [3]:
categories = {
    "pet cat": "宠物猫",
    "tomato": "番茄",
    "paper-cut": "剪纸",
    "computer": "电脑",
    "dumpling": "饺子"
}

def calculate_metrics(TP, FP, FN):
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1


def verify_and_calculate_metrics(target_root_folder, categories):
    total_precision, total_recall, total_f1 = 0, 0, 0

    for category, keyword in categories.items():
        TP, FP, FN = 0, 0, 0

        category_folder = os.path.join(target_root_folder, category)
        prompt = f"Please determine if this is a photo about {category}? Answer yes or no"
        args.query = prompt

        for image_name in os.listdir(category_folder):
            image_path = os.path.join(category_folder, image_name)
            args.image_file = image_path

            output = get_eval_model_output(args)
            is_correct = "yes" in output.lower()

            # 根据图片名称判断真实类别
            actual_category = keyword if keyword in image_name else "其他"

            # 更新统计数据
            if is_correct and actual_category == keyword:
                TP += 1
            elif is_correct and actual_category != keyword:
                FP += 1

        # 计算并打印当前类别的Precision、Recall和F1 score
        FN = 200 - TP
        precision, recall, f1 = calculate_metrics(TP, FP, FN)
        total_precision += precision
        total_recall += recall
        total_f1 += f1
        print(f"Category: {category}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    # 计算并打印总体平均Precision、Recall和F1 score
    average_precision = total_precision / len(categories)
    average_recall = total_recall / len(categories)
    average_f1 = total_f1 / len(categories)
    print(f"Average Precision: {average_precision:.4f}, Average Recall: {average_recall:.4f}, Average F1: {average_f1:.4f}")


# 示例调用函数
target_root_folder = "/content/drive/MyDrive/target_data3"
verify_and_calculate_metrics(target_root_folder, categories)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.w

Category: pet cat, Precision: 0.9662, Recall: 1.0000, F1 Score: 0.9828
Category: tomato, Precision: 0.9949, Recall: 0.9750, F1 Score: 0.9848
Category: paper-cut, Precision: 0.9336, Recall: 0.9850, F1 Score: 0.9586


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Category: computer, Precision: 1.0000, Recall: 0.9150, F1 Score: 0.9556
Category: dumpling, Precision: 0.8383, Recall: 0.9850, F1 Score: 0.9057
Average Precision: 0.9466, Average Recall: 0.9720, Average F1: 0.9575


释放内存

In [ ]:
del model

In [ ]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()